In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F

# 画像のパス
img_path = "/mnt/ishida/datacollect/image/GL20230523091545_002300424053_000060003618001_page_0.jpg"
img = Image.open(img_path)
img_np = np.array(img)

def padding_image(img: np.array):
    """画像の周りを0でpadding

    Args:
        img (np.array): original image
    
    Returns:
        pad_img (np.array): padded image
    """

    # 元の画像サイズ
    original_height, original_width = img.shape[:2]

    # Padding後のサイズ
    new_width, new_height = 1400, 800

    # パディングするサイズを計算
    top_pad = (new_height - original_height) // 2
    bottom_pad = new_height - original_height - top_pad
    left_pad = (new_width - original_width) // 2
    right_pad = new_width - original_width - left_pad

    # 画像にパディングを追加
    pad_img = cv2.copyMakeBorder(
        img, 
        top_pad,
        bottom_pad, 
        left_pad, 
        right_pad, 
        cv2.BORDER_CONSTANT, value=[0, 0, 0]
    )

    return pad_img

# 色の反転
img_trans = cv2.bitwise_not(img_np)

# グレースケールに変換
img_trans = cv2.cvtColor(img_trans, cv2.COLOR_RGB2GRAY)

# cv2.GaussianBlurを適用 (カーネルサイズ15x15, σ=5)
img_trans = cv2.GaussianBlur(img_trans, (15, 15), 5) # ぼかす
_, img_trans = cv2.threshold(img_trans, 0, 255, cv2.THRESH_OTSU)

print(img_trans.shape)
img_trans = padding_image(img_trans)
print(img_trans.shape)

# resize(45, 25)
img_resize = cv2.resize(img_trans, (45, 25), interpolation=cv2.INTER_AREA)


# 画像を可視化 (元の画像、ガウスブラー適用後の画像)
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 元の画像
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')  # 軸を非表示にする

# ガウスブラー適用後の画像
axes[1].imshow(img_trans, cmap='gray')
axes[1].set_title('Blurred Image (cv2.GaussianBlur)')
axes[1].axis('off')  # 軸を非表示にする

# resize画像
axes[2].imshow(img_resize, cmap='gray')
axes[2].set_title('Annotation Input, (45, 25)')
axes[2].axis('off')  # 軸を非表示にする

plt.tight_layout()
plt.show()
